Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# 04. Train in a remote Linux VM
* Create Workspace
* Create `train.py` file
* Create (or attach) DSVM as compute resource.
* Upoad data files into default datastore
* Configure & execute a run in a few different ways
    - Use system-built conda
    - Use existing Python environment
    - Use Docker 
* Find the best model in the run

## Prerequisites
Make sure you go through the [00. Installation and Configuration](00.configuration.ipynb) Notebook first if you haven't.

In [1]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [2]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.6


In [3]:
%load_ext dotenv

## Initialize Workspace

Initialize a workspace object from persisted configuration.

In [4]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location[:1], ws.subscription_id[:1], sep='\n')

Found the config file in: /workspace/amlsdk/AMLSDKNotebooks/aml_config/config.json
ghiordanDockerPower001ws
ghiordanDockerPower001rsg
e
e


## Create Experiment

**Experiment** is a logical container in an Azure ML Workspace. It hosts run records which can include run metrics and output artifacts from your experiments.

In [5]:
experiment_name = 'train-on-remote-vm'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

Let's also create a local folder to hold the training script.

In [6]:
import os
script_folder = './vm-run'
os.makedirs(script_folder, exist_ok=True)

## Upload data files into datastore
Every workspace comes with a default datastore (and you can register more) which is backed by the Azure blob storage account associated with the workspace. We can use it to transfer data from local to the cloud, and access it from the compute target.

In [7]:
# get the default datastore
ds = ws.get_default_datastore()
print(ds.name, ds.datastore_type, ds.account_name[:2], ds.container_name[:2])

workspacefilestore AzureFile gh az


Load diabetes data from `scikit-learn` and save it as 2 local files.

In [8]:
from sklearn.datasets import load_diabetes
import numpy as np

training_data = load_diabetes()
np.save(file='./features.npy', arr=training_data['data'])
np.save(file='./labels.npy', arr=training_data['target'])

Now let's upload the 2 files into the default datastore under a path named `diabetes`:

In [9]:
ds.upload_files(['./features.npy', './labels.npy'], target_path='diabetes', overwrite=True)

Uploading ./features.npy
Uploading ./labels.npy
Uploaded ./labels.npy, 1 files out of an estimated total of 2
Uploaded ./features.npy, 2 files out of an estimated total of 2


$AZUREML_DATAREFERENCE_e53622c1cff146cc8e8bea9dee49ad4f

## View `train.py`

For convenience, we created a training script for you. It is printed below as a text, but you can also run `%pfile ./train.py` in a cell to show the file. Please pay special attention on how we are loading the features and labels from files in the `data_folder` path, which is passed in as an argument of the training script (shown later).

In [10]:
# copy train.py into the script folder
import shutil
shutil.copy('./train.py', os.path.join(script_folder, 'train.py'))

with open(os.path.join(script_folder, './train.py'), 'r') as training_script:
    print(training_script.read())

'./vm-run/train.py'

# Copyright (c) Microsoft. All rights reserved.
# Licensed under the MIT license.

import os
import argparse

from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from azureml.core.run import Run
from sklearn.externals import joblib

import numpy as np

os.makedirs('./outputs', exist_ok=True)
parser = argparse.ArgumentParser()
parser.add_argument('--data-folder', type=str,
                    dest='data_folder', help='data folder')
args = parser.parse_args()

print('Data folder is at:', args.data_folder)
print('List all files: ', os.listdir(args.data_folder))

X = np.load(os.path.join(args.data_folder, 'features.npy'))
y = np.load(os.path.join(args.data_folder, 'labels.npy'))

run = Run.get_context()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)
data = {"train": {"X": X_train, "y": y_train},
        "test": {"X": X_test, "y": y_test}}

# list of numbers

## Create Linux DSVM as a compute target

**Note**: If creation fails with a message about Marketplace purchase eligibilty, go to portal.azure.com, start creating DSVM there, and select "Want to create programmatically" to enable programmatic creation. Once you've enabled it, you can exit without actually creating VM.
 
**Note**: By default SSH runs on port 22 and you don't need to specify it. But if for security reasons you switch to a different port (such as 5022), you can specify the port number in the provisioning configuration object.

In [11]:
compute_target_name = 'ghiordanXRgpuvm'

if False:
    from azureml.core.compute import DsvmCompute
    from azureml.core.compute_target import ComputeTargetException

    try:
        dsvm_compute = DsvmCompute(workspace=ws, name=compute_target_name)
        print('found existing:', dsvm_compute.name)
    except ComputeTargetException:
        print('creating new.')
        dsvm_config = DsvmCompute.provisioning_configuration(vm_size="Standard_D2_v2")
        dsvm_compute = DsvmCompute.create(ws, name=compute_target_name, provisioning_configuration=dsvm_config)
        dsvm_compute.wait_for_completion(show_output=True)

## Attach an existing Linux DSVM
You can also attach an existing Linux VM as a compute target. The default port is 22.

In [12]:
%dotenv

from azureml.core.compute import ComputeTarget, RemoteCompute 
# if you want to connect using SSH key instead of username/password you can provide parameters private_key_file and private_key_passphrase 
# attached_dsvm_compute = RemoteCompute.attach(workspace=ws,
#                                              name='ghiordanXRgpuvm',
#                                              username=os.getenv('COMPUTE_CONTEXT_VM_USER_NAME'),
#                                              address=os.getenv('COMPUTE_CONTEXT_VM_FQDN'),
#                                              ssh_port=os.getenv('COMPUTE_CONTEXT_VM_SSH_PORT'),
#                                              password=os.getenv('COMPUTE_CONTEXT_VM_PWD'))


attach_config = RemoteCompute.attach_configuration(address=os.getenv('COMPUTE_CONTEXT_VM_FQDN'),
                                                   ssh_port=os.getenv('COMPUTE_CONTEXT_VM_SSH_PORT'),
                                                   username=os.getenv('COMPUTE_CONTEXT_VM_USER_NAME'),
                                                   password=os.getenv('COMPUTE_CONTEXT_VM_PWD')
                                                   # If using ssh key
                                                   #private_key_file="path_to_a_file",
                                                   #private_key_passphrase="some_key_phrase"
                                                  )
attached_dsvm_compute = ComputeTarget.attach(workspace=ws, name='ghiordanXRgpuvm', attach_configuration=attach_config)

attached_dsvm_compute.wait_for_completion(show_output=True)


SucceededProvisioning operation finished, operation "Succeeded"


## Configure & Run
First let's create a `DataReferenceConfiguration` object to inform the system what data folder to download to the copmute target.

In [13]:
from azureml.core.runconfig import DataReferenceConfiguration
dr = DataReferenceConfiguration(datastore_name=ds.name, 
                   path_on_datastore='diabetes', 
                   mode='download', # download files from datastore to compute target
                   overwrite=True)

Now we can try a few different ways to run the training script in the VM.

### Configure a Docker run with new conda environment on the VM
You can execute in a Docker container in the VM. If you choose this option, the system will pull down a base Docker image, build a new conda environment in it if you ask for (you can also skip this if you are using a customer Docker image when a preconfigured Python environment), start a container, and run your script in there. This image is also uploaded into your ACR (Azure Container Registry) assoicated with your workspace, an reused if your dependencies don't change in the subsequent runs.

In [14]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies


# Load the "cpu-dsvm.runconfig" file (created by the above attach operation) in memory
docker_run_config = RunConfiguration(framework="python")

# Set compute target to the Linux DSVM
docker_run_config.target = compute_target_name

# Use Docker in the remote VM
docker_run_config.environment.docker.enabled = True

# Use CPU base image from DockerHub
docker_run_config.environment.docker.base_image = azureml.core.runconfig.DEFAULT_CPU_IMAGE
print('Base Docker image is:', docker_run_config.environment.docker.base_image)

# set the data reference of the run coonfiguration
docker_run_config.data_references = {ds.name: dr}

# specify CondaDependencies obj
docker_run_config.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'])

Base Docker image is: mcr.microsoft.com/azureml/base:0.2.0


### Submit the Experiment
Submit script to run in the Docker image in the remote VM. If you run this for the first time, the system will download the base image, layer in packages specified in the `conda_dependencies.yml` file on top of the base image, create a container and then execute the script in the container.

In [15]:
from azureml.core import ScriptRunConfig
src = ScriptRunConfig(source_directory=script_folder, 
                      script='train.py', 
                      run_config=docker_run_config,
                      # pass the datastore reference as a parameter to the training script
                      arguments=['--data-folder', str(ds.as_download())])
run = exp.submit(config=src)

In [16]:
run.wait_for_completion(show_output=True)

RunId: train-on-remote-vm_1545441823110

Streaming azureml-logs/60_control_log.txt

Streaming log file azureml-logs/60_control_log.txt

Streaming azureml-logs/80_driver_log.txt

Downloaded diabetes/labels.npy, 1 files out of an estimated total of 3
Downloaded diabetes/features.npy, 2 files out of an estimated total of 3
Data folder is at: workspacefilestore/diabetes
List all files:  ['features.npy', 'labels.npy']
alpha is 0.00, and mse is 3424.32
alpha is 0.05, and mse is 3408.92
alpha is 0.10, and mse is 3372.65
alpha is 0.15, and mse is 3345.15
alpha is 0.20, and mse is 3325.29
alpha is 0.25, and mse is 3311.56
alpha is 0.30, and mse is 3302.67
alpha is 0.35, and mse is 3297.66
alpha is 0.40, and mse is 3295.74
alpha is 0.45, and mse is 3296.32
alpha is 0.50, and mse is 3298.91
alpha is 0.55, and mse is 3303.14
alpha is 0.60, and mse is 3308.70
alpha is 0.65, and mse is 3315.36
alpha is 0.70, and mse is 3322.90
alpha is 0.75, and mse is 3331.17
alpha is 0.80, and mse is 3340.02
alpha

{'runId': 'train-on-remote-vm_1545441823110',
 'target': 'ghiordanXRgpuvm',
 'status': 'Finalizing',
 'startTimeUtc': '2018-12-22T01:23:45.597801Z',
 'properties': {'azureml.runsource': 'experiment',
  'ContentSnapshotId': '2825de45-0c87-4739-8571-e77422aff451'},
 'runDefinition': {'Script': 'train.py',
  'Arguments': ['--data-folder', '$AZUREML_DATAREFERENCE_workspacefilestore'],
  'SourceDirectoryDataStore': None,
  'Framework': 0,
  'Communicator': 0,
  'Target': 'ghiordanXRgpuvm',
  'DataReferences': {'workspacefilestore': {'DataStoreName': 'workspacefilestore',
    'Mode': 'Download',
    'PathOnDataStore': 'diabetes',
    'PathOnCompute': None,
    'Overwrite': True}},
  'JobName': None,
  'AutoPrepareEnvironment': True,
  'MaxRunDurationSeconds': None,
  'NodeCount': 1,
  'Environment': {'Python': {'InterpreterPath': 'python',
    'UserManagedDependencies': False,
    'CondaDependencies': {'name': 'project_environment',
     'dependencies': ['python=3.6.2',
      {'pip': ['azure

### View run history details

In [17]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
train-on-remote-vm,train-on-remote-vm_1545441823110,azureml.scriptrun,Finalizing,Link to Azure Portal,Link to Documentation


### Find the best model

Now we have tried various execution modes, we can find the best model from the last run.

In [18]:
# get all metris logged in the run
run.get_metrics()
metrics = run.get_metrics()

{'alpha': [0.0,
  0.05,
  0.1,
  0.15000000000000002,
  0.2,
  0.25,
  0.30000000000000004,
  0.35000000000000003,
  0.4,
  0.45,
  0.5,
  0.55,
  0.6000000000000001,
  0.65,
  0.7000000000000001,
  0.75,
  0.8,
  0.8500000000000001,
  0.9,
  0.9500000000000001],
 'mse': [3424.316688213733,
  3408.91531225893,
  3372.6496278100326,
  3345.1496434741894,
  3325.294679467877,
  3311.556250928974,
  3302.673633401725,
  3297.6587339442026,
  3295.741064355809,
  3296.316884705675,
  3298.909605807062,
  3303.1400555275163,
  3308.7042707723226,
  3315.3568399622563,
  3322.8983149039614,
  3331.1656169285875,
  3340.0246620321604,
  3349.3646443486023,
  3359.0935697484424,
  3369.1347399130477]}

In [19]:
# find the index where MSE is the smallest
indices = list(range(0, len(metrics['mse'])))
min_mse_index = min(indices, key=lambda x: metrics['mse'][x])

print('When alpha is {1:0.2f}, we have min MSE {0:0.2f}.'.format(
    metrics['mse'][min_mse_index], 
    metrics['alpha'][min_mse_index]
))

When alpha is 0.40, we have min MSE 3295.74.


In [20]:
run.get_file_names()

['azureml-logs/60_control_log.txt',
 'outputs/ridge_0.05.pkl',
 'outputs/ridge_0.20.pkl',
 'outputs/ridge_0.75.pkl',
 'outputs/ridge_0.45.pkl',
 'outputs/ridge_0.70.pkl',
 'outputs/ridge_0.95.pkl',
 'outputs/ridge_0.35.pkl',
 'outputs/ridge_0.30.pkl',
 'outputs/ridge_0.25.pkl',
 'outputs/ridge_0.55.pkl',
 'outputs/ridge_0.60.pkl',
 'outputs/ridge_0.85.pkl',
 'outputs/ridge_0.50.pkl',
 'outputs/ridge_0.15.pkl',
 'outputs/ridge_0.00.pkl',
 'outputs/ridge_0.80.pkl',
 'outputs/ridge_0.90.pkl',
 'outputs/ridge_0.40.pkl',
 'outputs/ridge_0.65.pkl',
 'outputs/ridge_0.10.pkl',
 'azureml-logs/80_driver_log.txt',
 'driver_log',
 'azureml-logs/azureml.log']

We know the model ridge_0.40.pkl is the best performing model from the eariler queries. 
So we can register it with the workspace, or download locally

In [21]:
# supply a model name, and the full path to the serialized model file.
model = run.register_model(model_name='second_best_ridge_model', model_path='./outputs/ridge_0.10.pkl')

In [22]:
from azureml.core.model import Model

model_local_file_path = os.path.join(*(['.', '..', '..']+['ridge_0.40.pkl'])) 
model_local_file_path
run.download_file(name='./outputs/ridge_0.40.pkl', output_file_path=model_local_file_path)

'./../../ridge_0.40.pkl'

In [23]:
# show ridge_0.40.pkl exists locally
!ls -l ./../../

total 304
-rw-r--r-- 1 root root 277806 Dec 21 19:25 00.configuration.html
-rw-rw-r-- 1 1003 1003  15589 Dec 22 01:23 00.configuration.ipynb
drwxrwxr-x 3 1003 1003   4096 Nov 11 21:49 01.getting-started
drwxrwxr-x 3 1003 1003   4096 Nov 21 02:37 10.register-model-create-image-deploy-service
drwxr-xr-x 2 root root   4096 Nov 10 01:29 aml_config
-rw-r--r-- 1 root root    658 Dec 22 01:23 ridge_0.40.pkl


In [24]:
!jupyter nbconvert --to html 04.train-on-remote-vm.ipynb

[NbConvertApp] Converting notebook 04.train-on-remote-vm.ipynb to html
[NbConvertApp] Writing 301504 bytes to 04.train-on-remote-vm.html
